<a href="https://colab.research.google.com/github/taaha3244/knowlegde_graphs/blob/main/neo4j_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j pypdf pyvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.2 MB/s eta 0:00:00


In [4]:
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
import os
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from google.colab import userdata
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
import os

from langchain_community.graphs import Neo4jGraph
from langchain_community.document_loaders import PyPDFLoader
from google.colab import userdata



In [5]:
os.environ["NEO4J_URI"] = userdata.get('neo4j_url')
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = userdata.get('neo4j_password')
openai=userdata.get('openai_api_key')


In [6]:
graph = Neo4jGraph()

In [7]:
llm = ChatOpenAI(temperature=0, model_name="gpt-4-turbo",openai_api_key=openai)
llm_transformer = LLMGraphTransformer(llm=llm)

In [10]:
loader = PyPDFLoader("/content/Audition Alexandre VAN DAMME.pdf")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [11]:
graph_documents = llm_transformer.convert_to_graph_documents(docs)

In [12]:
print(f"Nodes:{graph_documents[0].nodes}")

Nodes:[Node(id='Fd.19.Cro.1939', type='Dossier'), Node(id='Fd.10.Rl.106627/2019', type='Dossier'), Node(id='Djsoc', type='Organization'), Node(id='Ocrc-Section Marchés Pub', type='Organization'), Node(id='Julien Moinil', type='Person'), Node(id='Alexandre Van Damme', type='Person'), Node(id='Herman Pieter Celestijn Maria Ghisla Van Holsbeeck', type='Person'), Node(id='Christophe Jean-Paul Luc Henrotay', type='Person'), Node(id='Johan Marie Jozef Jacques Van Biesbroeck', type='Person'), Node(id='1070 Anderlecht, Avenue Théo Verbeeck 2 - 1', type='Location'), Node(id='Forgery', type='Crime'), Node(id='Internet-Free Fraud', type='Crime'), Node(id='Criminal Association', type='Crime')]


In [13]:
print(f"Relationships:{graph_documents[0].relationships}")

Relationships:[Relationship(source=Node(id='Julien Moinil', type='Person'), target=Node(id='Djsoc', type='Organization'), type='MEMBER'), Relationship(source=Node(id='Forgery', type='Crime'), target=Node(id='1070 Anderlecht, Avenue Théo Verbeeck 2 - 1', type='Location'), type='OCCURRED_AT'), Relationship(source=Node(id='Internet-Free Fraud', type='Crime'), target=Node(id='1070 Anderlecht, Avenue Théo Verbeeck 2 - 1', type='Location'), type='OCCURRED_AT'), Relationship(source=Node(id='Criminal Association', type='Crime'), target=Node(id='1070 Anderlecht, Avenue Théo Verbeeck 2 - 1', type='Location'), type='OCCURRED_AT')]


In [14]:
graph.add_graph_documents(graph_documents)

In [15]:
from neo4j import GraphDatabase

# Connect to your Neo4j database
uri = userdata.get('neo4j_url')
user = "neo4j"
password = userdata.get('neo4j_password')

driver = GraphDatabase.driver(uri, auth=(user, password))

def get_nodes_and_relationships():
    query = """
    MATCH (n)
    OPTIONAL MATCH (n)-[r]->(m)
    RETURN n, r, m
    """

    with driver.session() as session:
        result = session.run(query)
        nodes = {}
        relationships = []
        for record in result:
            node = record["n"]
            node_id = node.id
            node_label = list(node.labels)[0]
            node_properties = node["id"]

            if node_id not in nodes:
                nodes[node_id] = {"label": node_properties, "group": node_label}

            relationship = record["r"]
            if relationship is not None:
                relationships.append(relationship)

        return nodes, relationships

# Retrieve nodes and relationships
nodes, relationships = get_nodes_and_relationships()

# Close the driver after use
driver.close()

# Print the retrieved data for verification
print("Nodes:")
for node_id, props in nodes.items():
    print(f"Node ID: {node_id}, Label: {props['label']}, Group: {props['group']}")

print("\nRelationships:")
for relationship in relationships:
    print(f"Relationship: {relationship}")

<ipython-input-15-b23375ff6f1f>:23: DeprecationWarning: `id` is deprecated, use `element_id` instead
  node_id = node.id


Nodes:
Node ID: 0, Label: Myriam Lebrun, Group: Person
Node ID: 1, Label: Sa Rsca, Group: Organization
Node ID: 2, Label: Roger Vanden Stock, Group: Person
Node ID: 3, Label: Herman Van Holsbeeck, Group: Person
Node ID: 4, Label: Christophe Henrotay, Group: Person
Node ID: 5, Label: Etienne Davignon, Group: Person
Node ID: 6, Label: Monsieur Ousmanov, Group: Person
Node ID: 7, Label: Ocrc-Section Marchés Publics, Group: Organization
Node ID: 8, Label: Rue Royale 202 A, Group: Location
Node ID: 9, Label: 1000 Bruxelles, Group: Location
Node ID: 10, Label: Dupont Antoine, Group: Person
Node ID: 11, Label: Lebrun Myriam, Group: Person
Node ID: 12, Label: Van Damme Alexandre Albert Marcello, Group: Person
Node ID: 13, Label: Neuilly Sur Seine, Group: Location
Node ID: 14, Label: France, Group: Location
Node ID: 15, Label: Office Central Pour La Répression De La Corruption, Group: Organization
Node ID: 16, Label: Police Judiciaire Fédérale, Group: Organization
Node ID: 17, Label: Nv Alychlo

In [17]:
from pyvis.network import Network

# Initialize the network with a larger canvas size
net = Network(notebook=True, directed=True,cdn_resources='remote', height='800px', width='100%')

# Enable physics for better node distribution
net.barnes_hut()

def add_nodes_and_edges(net, nodes, relationships):
    # Add nodes
    for node_id, props in nodes.items():
        net.add_node(node_id, label=props["label"], title=props["label"], group=props["group"])
        print(f"Added node: {node_id} [{props['label']}]")

    # Add edges
    for relationship in relationships:
        source = relationship.start_node.id
        target = relationship.end_node.id
        rel_type = relationship.type

        net.add_edge(source, target, title=rel_type, label=rel_type, arrows='to', arrowStrikethrough=False)
        print(f"Added edge: {source} -> {target} [{rel_type}]")

# Add nodes and edges to the network
add_nodes_and_edges(net, nodes, relationships)

# Display the network directly in the notebook
net.show("graph.html")

Added node: 0 [Myriam Lebrun]
Added node: 1 [Sa Rsca]
Added node: 2 [Roger Vanden Stock]
Added node: 3 [Herman Van Holsbeeck]
Added node: 4 [Christophe Henrotay]
Added node: 5 [Etienne Davignon]
Added node: 6 [Monsieur Ousmanov]
Added node: 7 [Ocrc-Section Marchés Publics]
Added node: 8 [Rue Royale 202 A]
Added node: 9 [1000 Bruxelles]
Added node: 10 [Dupont Antoine]
Added node: 11 [Lebrun Myriam]
Added node: 12 [Van Damme Alexandre Albert Marcello]
Added node: 13 [Neuilly Sur Seine]
Added node: 14 [France]
Added node: 15 [Office Central Pour La Répression De La Corruption]
Added node: 16 [Police Judiciaire Fédérale]
Added node: 17 [Nv Alychlo]
Added node: 18 [Maître Marc Preumont]
Added node: 19 [Van Damme Alexandre Albert Marcelio Joseph]
Added node: 20 [Sa Cesto]
Added node: 21 [Johan Van Biesbroeck]
Added node: 22 [Bvba Jovb]
Added node: 23 [Interbrew]
Added node: 24 [Inbev]
Added node: 25 [Abi]
Added node: 26 [Djsoc]
Added node: 27 [Ocrc-Section Marchés Pub]
Added node: 28 [Julien

<ipython-input-17-4b1e13c85b11>:17: DeprecationWarning: `id` is deprecated, use `element_id` instead
  source = relationship.start_node.id
<ipython-input-17-4b1e13c85b11>:18: DeprecationWarning: `id` is deprecated, use `element_id` instead
  target = relationship.end_node.id


In [18]:
from IPython.core.display import display, HTML
display(HTML("graph.html"))